In [26]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error
import pickle

In [2]:
def clean_df(df_link):
    df = pd.read_parquet(df_link)
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)
    df.drop(df.loc[df.duration < 0].index, inplace=True)
    df = df[(df.duration > 1) & (df.duration <= 60)].copy()
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

In [3]:
df_train = clean_df('../data/yellow_tripdata_2023-01.parquet')
df_val = clean_df('../data/yellow_tripdata_2023-02.parquet')

In [4]:
display(df_train.shape[0])
display(df_val.shape[0])

3008892

2855630

In [5]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [6]:
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [7]:
y_train = df_train['duration'].values
y_val = df_val['duration'].values

In [28]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.816896382604758

In [27]:
lasso = Lasso(0.01)
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

8.136356037251318

In [29]:
rid = Ridge()
rid.fit(X_train, y_train)

y_pred = rid.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.811288296965185

In [23]:
model = ElasticNet(alpha=0.1, l1_ratio=0.5)  # Parámetros alpha y l1_ratio
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

9.223182097109754

In [30]:
with open('../models/ridge_lreg.bin', 'wb') as f_out:
    pickle.dump((dv, rid), f_out)